In [5]:
from selenium import webdriver
import time
import pandas as pd
import re
from selenium.webdriver.common.by import By
#from datetime import datetime, date, timedelta
from selenium.webdriver.chrome.options import Options
import requests
import json

from webdriver_manager.chrome import ChromeDriverManager
import mysql
import mysql.connector
import os
from dotenv import load_dotenv
import datetime


In [20]:


load_dotenv(".env")


print('ライブラリの読み込み完了')

def insert_data_bulk(df,cnx):
    insert_sql = """INSERT INTO mysql_database.schedule (都道府県, イベント日, 曜日, 店舗名, 取材名, 媒体名, 取材ランク,取得時間) values (%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    print("Insert bulk data")


def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text



def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 


def login_scraping_site(area_name):
    options = Options()
    #options.add_argument('--headless')
    options.add_argument("--no-sandbox")

    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install() 
    browser.implicitly_wait(10)
    url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
    #admageを開く
    browser.get(url_login)
    browser.implicitly_wait(10)

    # id
    element = browser.find_element(By.NAME,"id")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_ID'))

    # pw
    element = browser.find_element(By.NAME,"pass")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_PW'))

    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
    browser.get(url)
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
    browser.get(url)
    browser.implicitly_wait(10)
    return browser


furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','イベント日','店舗名','取材名','媒体名','取材ランク'])
browser = login_scraping_site('chubu')
elements = browser.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
i = 0
while True:
    browser.find_element(By.CLASS_NAME,"head_change_main").click()
    browser.implicitly_wait(10)
    if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
        browser = login_scraping_site('chubu')
    else:
        pass
    try:
        elements = browser.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
        baitai_name = elements[i].text.split(' ')[0]
        baitai_name = baitai_name.replace('パチマガスロマガじゃ…','パチマガスロマガ').replace('パチンコ店長のホール…','パチンコ店長のホール攻略')

        if  ('県' in baitai_name) or ('府' in baitai_name) or ('東京都' in baitai_name) or ('北海道' in baitai_name):
            print('県がついているためパスを処理しました',baitai_name)
            i += 1
            continue

        print(baitai_name)
        elements[i].click()
        # if 'ホールナビ' in baitai_name:
        #     print(baitai_name)
        #     break
        while True:
            for syuzai_tenpo_data in browser.find_elements(By.CLASS_NAME,"osbox"):
                tenpo_name = syuzai_tenpo_data.find_element(By.CLASS_NAME,"oslh2").text
                #print(tenpo_name.text)
                syuzai_date = syuzai_tenpo_data.find_element(By.CLASS_NAME,"oslmd").text
                rank_and_syuzai_name = syuzai_tenpo_data.find_element(By.CLASS_NAME,"list_event_name").text
                syuzai_rank = rank_and_syuzai_name.split('\n')[0]
                syuzai_name = rank_and_syuzai_name.split('\n')[1]
                prefectures = syuzai_tenpo_data.find_elements(By.CLASS_NAME,"oslha")[0].text
                #print(baitai_name,syuzai_date ,syuzai_rank,syuzai_name,prefectures)#prefectures
                record = pd.Series([prefectures,syuzai_date, tenpo_name,syuzai_name,baitai_name,syuzai_rank], index=furture_syuzai_list_df.columns)
                furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
                #print(record)
                #break
                browser.implicitly_wait(10)
            if browser.find_element(By.CLASS_NAME,'navi_1_next').text == '次へ':
                browser.find_element(By.CLASS_NAME,'navi_1_next').click()
            else:
                break
        i += 1
        # time.sleep(1)
        #break
        #break
    except Exception as e:
        print('エラー理由',e)
        break
    # if i > 13:
    break

#browser.quit()


ライブラリの読み込み完了


C:\Users\tsc95\AppData\Local\Temp\ipykernel_35888\2701365648.py:36: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install()


県がついているためパスを処理しました 愛知県
県がついているためパスを処理しました 静岡県
県がついているためパスを処理しました 山梨県
県がついているためパスを処理しました 長野県
県がついているためパスを処理しました 岐阜県
県がついているためパスを処理しました 新潟県
県がついているためパスを処理しました 富山県
県がついているためパスを処理しました 石川県
県がついているためパスを処理しました 福井県
県がついているためパスを処理しました 三重県
県がついているためパスを処理しました 愛知県
県がついているためパスを処理しました 静岡県
県がついているためパスを処理しました 山梨県
県がついているためパスを処理しました 長野県
県がついているためパスを処理しました 岐阜県
県がついているためパスを処理しました 新潟県
県がついているためパスを処理しました 富山県
県がついているためパスを処理しました 石川県
県がついているためパスを処理しました 福井県
県がついているためパスを処理しました 三重県
APチャンネル


C:\Users\tsc95\AppData\Local\Temp\ipykernel_35888\2701365648.py:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_35888\2701365648.py:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_35888\2701365648.py:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_35888\2701365648.py:107: FutureWarning: The frame.append method is deprecated and will be removed 

In [46]:
pattern = '東京都|北海道|(京都|大阪)府|.{2,3}県'
# 都道府県を抽出する
furture_syuzai_list_df = furture_syuzai_list_df[furture_syuzai_list_df['都道府県'] != '']
# furture_syuzai_list_df['都道府県']=furture_syuzai_list_df['都道府県'].apply(lambda x:re.match(pattern,x).group())
prefectures_list = []
for adress in furture_syuzai_list_df['都道府県']:
    adress = adress.split(']')[-1]
    try:
        #print(re.match(pattern,adress).group())
        prefectures_list.append(re.match(pattern,adress).group())
    except:
        prefectures_list.append('情報なし')
furture_syuzai_list_df['都道府県'] = prefectures_list
furture_syuzai_list_df_1 = furture_syuzai_list_df
furture_syuzai_list_df_1 = pd.concat([furture_syuzai_list_df_1, furture_syuzai_list_df_1['イベント日'].str.split('(', expand=True)], axis=1).drop('イベント日', axis=1)
furture_syuzai_list_df_1.rename(columns={0: 'イベント日', 1: '曜日'}, inplace=True)
furture_syuzai_list_df_1['曜日'] = furture_syuzai_list_df_1['曜日'].map(lambda x:x.replace(')',''))
furture_syuzai_list_df_1['イベント日'] = pd.to_datetime(furture_syuzai_list_df_1['イベント日'])
furture_syuzai_list_df_1 = furture_syuzai_list_df_1 [['都道府県','イベント日','曜日',	'店舗名','取材名','媒体名','取材ランク']]
furture_syuzai_list_df_1

NameError: name 'furture_syuzai_list_df' is not defined

In [16]:
cnx = mysql.connector.connect(
                        user = os.getenv('DB_USER_NAME'),
                        password=os.getenv('DB_PASSWORD'), 
                        host=os.getenv('DB_HOST'), 
                        port='3306',
                        database=os.getenv('DB_NAME'))

cursor = cnx.cursor(buffered=True)

In [47]:
# 現在時刻
today = datetime.datetime.now()
print(today)
#[結果] 2021-08-23 07:12:20.806648
# 1日後
today_str:str = today.strftime('%Y-%m-%d')
eight_days_after:str = (today + datetime.timedelta(days=8)).strftime('%Y-%m-%d')
yesterday:str = (today + datetime.timedelta(days=-2)).strftime('%Y-%m-%d')
#### Create dataframe from resultant table ####
cnx = mysql.connector.connect(
                        user = os.getenv('DB_USER_NAME'),
                        password=os.getenv('DB_PASSWORD'), 
                        host=os.getenv('DB_HOST'), 
                        port='3306',
                        database=os.getenv('DB_NAME'))

cursor = cnx.cursor()


prefectures ="("
prefecture_list = ['愛知県','静岡県','山梨県','長野県','岐阜県','新潟県','富山県','石川県','福井県','三重県']
for i,text in enumerate(prefecture_list):
    if i == (len(prefecture_list) -1 ):
        prefectures += f"'{text}'"
    else:
        prefectures += f"'{text}',"
prefectures += ')'
print(prefectures)


sql = f"""
        SELECT *
        FROM schedule
        WHERE イベント日 BETWEEN '{yesterday}' AND '{eight_days_after}'
        AND  都道府県 in {prefectures}
        """
print(sql)
cursor.execute(sql)
#cols = [col[0] for col in cursor.description]
sql_syuzai_report_all_df = pd.DataFrame(cursor.fetchall(),columns = ['id','都道府県','イベント日','曜日','店舗名','取材名','媒体名','取材ランク','取得時間'])
sql_syuzai_report_all_df.sort_values('イベント日')
#display(sql_syuzai_report_all_df)
sql_syuzai_report_all_df = sql_syuzai_report_all_df[['都道府県','イベント日','曜日','店舗名','取材名','媒体名','取材ランク',]]
#display(sql_syuzai_report_all_df)
# cursor = cnx.cursor()
# # #### Connetion Established ####
# # #### Execute query ####
# query1 = ("select * from 
# _gcp")
# cursor.execute(query1)
# #### Create dataframe from resultant table ####
display(sql_syuzai_report_all_df)

concat_df = pd.concat([sql_syuzai_report_all_df,furture_syuzai_list_df_1])
concat_df['イベント日'] = pd.to_datetime(concat_df['イベント日'])
concat_df = concat_df[~concat_df.duplicated(keep=False)]

concat_df['取得時間'] = datetime.datetime.now()
concat_df

2023-03-18 04:10:55.706315
('愛知県','静岡県','山梨県','長野県','岐阜県','新潟県','富山県','石川県','福井県','三重県')

        SELECT *
        FROM schedule
        WHERE イベント日 BETWEEN '2023-03-16' AND '2023-03-26'
        AND  都道府県 in ('愛知県','静岡県','山梨県','長野県','岐阜県','新潟県','富山県','石川県','福井県','三重県')
        


,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク
0,長野県,2023-03-16,木,チャンピオンANNEX,APチャンネル来店収録,APチャンネル,B
1,三重県,2023-03-16,木,サン亀山店,ジャックレポート,APチャンネル,・
2,三重県,2023-03-17,金,サン亀山店,沖花レポート,APチャンネル,・
3,長野県,2023-03-18,土,ニューアサヒ千曲店,APチャンネル来店,APチャンネル,C
4,岐阜県,2023-03-18,土,ウイング池田南店,沖花レポート,APチャンネル,・
...,...,...,...,...,...,...,...
3095,愛知県,2023-03-24,金,ミカド観光津島店,旧イベ(4のつく日),旧イベ,・
3096,三重県,2023-03-24,金,FUJI津店,旧イベ(毎月 24日),旧イベ,・
3097,三重県,2023-03-24,金,富士井田川店,旧イベ(毎月 24日),旧イベ,・
3098,福井県,2023-03-24,金,NEO ZOO大和田店,旧イベ(4のつく日),旧イベ,・


NameError: name 'furture_syuzai_list_df_1' is not defined

In [37]:
insert_data_bulk(concat_df ,cnx)
cnx.commit()

post_line_text(f'{len(concat_df )}件の東海の取材予定追加おわり',os.getenv('LINE_TOKEN'))


Insert bulk data


In [20]:
import pandas as pd
syuzai_table_df = pd.read_csv(r'csv\取材まとめ.csv')
syuzai_table_df_1 = syuzai_table_df[syuzai_table_df['baitai_name'] != '未調査']
syuzai_table_df_1

,syuzai_name,baitai_name,kouyaku
0,1日回胴録ハンチョウ石神収録 ガット石神,1GAME,来店実践
1,いきなりヨースケ収録 ヨースケ,1GAME,未調査
2,来店実戦_BASHtv,BASHtv,来店実践
3,BASHFIGHT,BASHtv,未調査
4,据膳上膳,BASHtv,高設定据置示唆
...,...,...,...
338,実践収録,不明,誰かの来店
339,実践来店,不明,誰かの来店
340,並ばせ屋が診る!,並ばせ屋,全台設定⑥を2機種\n もしくは2箇所（列）
2042,潜入取材ギガテン,スロパチ,未調査


In [27]:
syuzai_table_df_2 = syuzai_table_df_1.replace({'スロパチ': 'スロパチステーション', 'DMM': 'DMMぱちタウン','スロマガ':'パチマガスロマガ','ホール攻略':'パチンコ店長のホール攻略','ジャンバリ':'ジャンバリ.TV','天草ヤスヲ':'天草ヤスヲのちゅんげーリサーチ'})
syuzai_table_df_2.columns = ['取材名','媒体名','公約']
syuzai_table_df_2

,取材名,媒体名,公約
0,1日回胴録ハンチョウ石神収録 ガット石神,1GAME,来店実践
1,いきなりヨースケ収録 ヨースケ,1GAME,未調査
2,来店実戦_BASHtv,BASHtv,来店実践
3,BASHFIGHT,BASHtv,未調査
4,据膳上膳,BASHtv,高設定据置示唆
...,...,...,...
338,実践収録,不明,誰かの来店
339,実践来店,不明,誰かの来店
340,並ばせ屋が診る!,並ばせ屋,全台設定⑥を2機種\n もしくは2箇所（列）
2042,潜入取材ギガテン,スロパチステーション,未調査


In [22]:
syuzai_table_df_1['baitai_name'].unique()

array(['1GAME', 'BASHtv', 'DMM', 'dorislo', 'Gooパチ', 'hippo',
       'IMCいむちゃんねる', 'P-martTV', 'SEVENSTV', 'アツ姫', 'アメスロ', 'オフミー',
       'ジャンバリ', 'スクープTV', 'スロット日報', 'スロパチ', 'スロマガ', 'ゼロノス', 'チーム夢戦',
       'でちゃう', 'ニコナナ', 'パチ7', 'パチスロ熱盛(@zPLie1cOUYtF6sW)', 'パチスロ必勝本',
       'フリー', 'ホールサーチマン', 'ホール攻略', 'リーチエンジェル', '一撃', '一撃&DMM', '一撃✖️DMM',
       '回胴連', '娯楽の殿堂', '新装', '対抗', '天草ヤスヲ', '日雇いホール', '日雇いホルアド', '爆ガチ！',
       '爆スロ', '不明', '並ばせ屋'], dtype=object)

In [21]:
sql_syuzai_report_all_df_1['媒体名'].unique()

array(['APチャンネル', 'ChiiiPS(チップス)', 'DMMぱちタウン', 'PS-EMOTION',
       'あかだ主任のガチンコ…', 'その他取材・企画', 'でちゃう！', 'ななライズ', 'アツ姫', 'アナコンダ',
       'アメスロ', 'エースプロ', 'オフミー', 'ガチアン', 'グーパチ', 'ゲッツ!!', 'ジャンバリ.TV',
       'スクープTV', 'スパ満開WEB', 'スロパチステーション', 'ダブルジャッジ', 'ネクストゾーン',
       'パチマガスロマガ', 'パチンコウォーカー', 'パチンコ店長のホール攻略', 'ホールサーチマン', 'ホールナビ',
       'リアルガチホール', 'リベンジャーズ', '五神獣', '名探偵ショウゴローの…', '旧イベ', '新台入替'],
      dtype=object)

In [26]:
sql_syuzai_report_all_df_1 = sql_syuzai_report_all_df[['取材名','媒体名']]
sql_syuzai_report_all_df_1 = sql_syuzai_report_all_df_1[~sql_syuzai_report_all_df_1.duplicated(keep='last')]
sql_syuzai_report_all_df_1['公約'] = None
sql_syuzai_report_all_df_1

,取材名,媒体名,公約
0,ぱちレポA,APチャンネル,None
2,APチャンネル来店収録,APチャンネル,None
3,ジャックレポート,APチャンネル,None
5,APチャンネル来店,APチャンネル,None
8,ガチレポエース(光),APチャンネル,None
...,...,...,...
2550,旧イベ(毎月 22日),旧イベ,None
2551,旧イベ(2のつく日),旧イベ,None
2552,旧イベ(毎週 水曜日),旧イベ,None
2559,旧イベ(ゾロ目の日),旧イベ,None


In [45]:
concat_df =pd.concat([syuzai_table_df_2,sql_syuzai_report_all_df_1],axis=0)
#concat_df.sort_values('取材名').to_csv('取材.csv',index=False)
concat_df[~concat_df.duplicated(subset=['取材名'])].sort_values(['取材名','公約'],ascending=False).to_csv('取材公約用マスター.csv',index=False)

In [48]:
sql_syuzai_report_all_df

,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク
0,長野県,2023-03-16,木,チャンピオンANNEX,APチャンネル来店収録,APチャンネル,B
1,三重県,2023-03-16,木,サン亀山店,ジャックレポート,APチャンネル,・
2,三重県,2023-03-17,金,サン亀山店,沖花レポート,APチャンネル,・
3,長野県,2023-03-18,土,ニューアサヒ千曲店,APチャンネル来店,APチャンネル,C
4,岐阜県,2023-03-18,土,ウイング池田南店,沖花レポート,APチャンネル,・
...,...,...,...,...,...,...,...
3095,愛知県,2023-03-24,金,ミカド観光津島店,旧イベ(4のつく日),旧イベ,・
3096,三重県,2023-03-24,金,FUJI津店,旧イベ(毎月 24日),旧イベ,・
3097,三重県,2023-03-24,金,富士井田川店,旧イベ(毎月 24日),旧イベ,・
3098,福井県,2023-03-24,金,NEO ZOO大和田店,旧イベ(4のつく日),旧イベ,・


In [50]:
convert_parlar_name_df = pd.read_csv(r'csv\店舗名変換テーブル.csv')
convert_parlar_name_df = convert_parlar_name_df.rename(columns={'ホールナビ店舗名':'店舗名'})
convert_parlar_name_df 

,店舗kintone,経度,緯度,DMM店舗名,住所,shop_id,アナスロ店舗名,都道府県,店舗名
0,P01378,141.514724,43.085828,アミューズメントパーラー吉兆,北海道江別市大麻泉町47-37,1378,アミューズメントパーラー吉兆,北海道,アミューズメントパーラー吉兆
1,P01582,143.207331,42.961369,音更ひまわり,北海道河東郡音更町木野大通東13丁目1番21,1582,ひまわり音更店,北海道,ひまわり音更店
2,P01351,141.292820,43.106287,EBH,北海道札幌市西区発寒15条4-13-1,13160,EBH,北海道,EBH
3,P02562,140.477010,36.365522,キコーナ水戸駅南店,茨城県水戸市城南1-4-1,2562,キコーナ水戸駅南店,茨城県,キコーナ水戸駅南店
4,P04933,139.465162,35.451103,P’Stown桜ヶ丘,神奈川県大和市福田1-6-10,4933,ピーズ・タウン桜ヶ丘,神奈川県,ピーズ・タウン桜ヶ丘
...,...,...,...,...,...,...,...,...,...
2352,P00363,139.699800,35.658285,楽園渋谷駅前店,東京都渋谷区道玄坂1-3-1,363,楽園渋谷駅前店,東京都,楽園渋谷駅前店
2353,P12714,138.553588,35.631724,マルハン甲府店,山梨県甲府市国母8-20-10,13138,マルハン甲府店,山梨県,マルハン甲府店
2354,P12738,139.600920,35.703498,メッセ西荻窪店,東京都杉並区西荻南3-14-7,13170,メッセ西荻窪店,東京都,メッセ西荻窪店
2355,P12792,NaN,NaN,マルキ戸田店,埼玉県戸田市美女木東1-3-2,13227,マルキ戸田店,埼玉県,マルキ戸田店


In [53]:
pledge_df = pd.read_csv(r'csv\取材公約用マスター.csv')
pledge_df

,取材名,媒体名,公約内容
0,ｙｔｒ来店実践,SEVENSTV,来店実践
1,Ａレポート,アツ姫,Aタイプに高設定投入
2,黒バラの極意(来店ver),ジャンバリ.TV,NaN
3,鮮魚レポート,アメスロ,NaN
4,魔法少女レポート スロット取材,爆ガチ！,まどかマギカシリーズが狙い目
...,...,...,...
752,AceOVER,アツ姫,NaN
753,APチャンネル来店収録,APチャンネル,NaN
754,APチャンネル来店,APチャンネル,NaN
755,1日回胴録ハンチョウ石神収録 ガット石神,1GAME,来店実践


In [62]:
tenpo_syuzai_merge_df = pd.merge(sql_syuzai_report_all_df, convert_parlar_name_df, on='店舗名')
pmerge_df = pd.merge(tenpo_syuzai_merge_df, pledge_df, on='取材名')
pmerge_df[pmerge_df['都道府県_x'] == '愛知県'].sort_values('イベント日')

,都道府県_x,イベント日,曜日,店舗名,取材名,媒体名_x,取材ランク,店舗kintone,経度,緯度,DMM店舗名,住所,shop_id,アナスロ店舗名,都道府県_y,媒体名_y,公約内容
300,愛知県,2023-03-16,木,中日ホール竹谷店,旧イベ(6のつく日),旧イベ,・,P06135,137.212806,34.830467,中日ホール竹谷店,愛知県蒲郡市竹谷町下日山2-1,6135,中日ホール竹谷店,愛知県,旧イベ,NaN
285,愛知県,2023-03-16,木,キャッスル岩倉店,旧イベ(6のつく日),旧イベ,・,P12439,136.864690,35.276831,キャッスル岩倉店,愛知県岩倉市中央町3-79,12812,キャッスル岩倉店,愛知県,旧イベ,NaN
280,愛知県,2023-03-16,木,キング666東海店,旧イベ(6のつく日),旧イベ,・,P12358,136.912161,35.020009,キング666東海店,愛知県東海市富木島町伏見一丁目18番8,12721,キング666東海店,愛知県,旧イベ,NaN
279,愛知県,2023-03-16,木,キング666東海店,旧イベ(6のつく日),旧イベ,・,P12358,136.912161,35.020009,キング666東海店,愛知県東海市富木島町伏見一丁目18番8,12721,キング666東海店,愛知県,旧イベ,NaN
278,愛知県,2023-03-16,木,キング666一宮店,旧イベ(6のつく日),旧イベ,・,P05949,136.806394,35.320017,キング666一宮店,愛知県一宮市西島町2-11-1,5949,キング666一宮店,愛知県,旧イベ,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,愛知県,2023-03-24,金,ラッキープラザ名古屋西インター七宝店,光るん×スロッター,パチンコ店長のホール攻略,・,P12265,136.801224,35.156045,ラッキープラザ名古屋西インター七宝店,愛知県あま市七宝町伊福苗代126,12596,ラッキープラザ名古屋西インター七宝店,愛知県,パチンコ店長のホール攻略,NaN
149,愛知県,2023-03-24,金,マルハン小牧店,光るん×スロッター,パチンコ店長のホール攻略,・,P06178,136.913237,35.304025,マルハン小牧店,愛知県小牧市大字村中字巾下１９９４番地,6178,マルハン小牧店,愛知県,パチンコ店長のホール攻略,NaN
1344,愛知県,2023-03-24,金,キクヤ春日井店,ナビ子AI予想【差枚プラス】★5 (毎週 金曜日),ホールナビ,S,P05992,136.959215,35.270396,キクヤ春日井店,愛知県春日井市町屋町字黒福3736,5992,キクヤ春日井店,愛知県,ホールナビ,NaN
143,愛知県,2023-03-24,金,マルハン新守山駅前店,光るん×スロッター,パチンコ店長のホール攻略,・,P05825,136.949377,35.206531,マルハン新守山駅前店,愛知県名古屋市守山区新守西１５０１番地,5825,マルハン新守山駅前店,愛知県,パチンコ店長のホール攻略,NaN
